In [7]:
import pandas as pd
import json
from openai import OpenAI
from tqdm import tqdm
import time
import os

In [3]:
def translate_item(client, item, index, pbar=None):
    """단일 항목을 번역하는 함수"""
    try:
        # 번역 요청 구성
        system_prompt = """당신은 영어를 한국어로 번역하는 전문가입니다.
        다음은 지하철역의 각종 업무에 대한 질문과 답변 데이터입니다.
주어진 영어 텍스트를 자연스러운 한국어로 번역해주세요.
전문 용어나 고유명사는 적절히 처리해주세요."""
        user_prompt = f"""다음 영어 텍스트를 한국어로 번역해주세요:
        출력은 JSON 형식으로만 추출해야하며 입력과 동일한 양식으로 출력해주세요.
[입력]
{json.dumps(item, ensure_ascii=False)}
번역 결과만 제공해주세요."""

        # 로그 출력
        print(f"\n[번역 시작 - 항목 {index}]")
        start_time = time.time()

        # API 호출로 번역
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.1,
            response_format={"type": "json_object"},
        )

        # 번역 결과 적용
        translated_text = response.choices[0].message.content.strip()
        translated_item = json.loads(translated_text)

        # 소요시간 및 완료 로그
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"[번역 완료 - 항목 {index}] ({elapsed_time:.2f}초 소요)")

        # 진행바 업데이트
        if pbar:
            pbar.update(1)
            pbar.set_description(f"처리 중: {index}")

        return translated_item

    except Exception as e:
        error_msg = f"Error occurred at index {index}: {e}"
        print(f"\n[에러 - 항목 {index}] {error_msg}")

        # 진행바 업데이트
        if pbar:
            pbar.update(1)
            pbar.set_description(f"에러 발생: {index}")

        return None


def translate_english_to_korean(data_list):
    """
    List[dict] 형태의 데이터에서 영어로 된 'user_input'과 'reference' 필드를 한국어로 번역합니다.
    각 항목을 개별적으로 API 호출하여 번역합니다.
    Args:
        data_list: 번역할 List[dict] 형태의 데이터
    Returns:
        번역된 데이터가 포함된 List[dict]
    """
    # OpenAI 클라이언트 초기화
    client = OpenAI()

    # 결과 저장 리스트
    translated_data = []

    # 총 항목 수와 처리 현황 계산 변수
    total_items = len(data_list)
    success_count = 0
    error_count = 0

    print(f"\n[번역 시작] 총 {total_items}개 항목 처리 예정")

    # tqdm으로 진행상황 표시
    with tqdm(total=total_items, desc="번역 진행률") as pbar:
        for index, item in enumerate(data_list):
            # 항목 번역
            translated_item = translate_item(client, item, index, pbar)

            # 결과 처리
            if translated_item:
                translated_data.append(translated_item)
                success_count += 1
            else:
                error_count += 1

            print(translated_item)

            # 10개 항목마다 중간 상황 보고
            if (index + 1) % 10 == 0 or index == total_items - 1:
                print(
                    f"\n[진행 상황] {index + 1}/{total_items} 완료 (성공: {success_count}, 실패: {error_count})"
                )

    # 최종 결과 요약
    print(
        f"\n[번역 완료] 총 {total_items}개 중 {success_count}개 성공, {error_count}개 실패"
    )

    return translated_data


def main():
    # 예시 데이터 - List[dict] 형태
    file_paths = [
        "/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset_100.csv",
        "/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset_ko_en_mixed.csv",
        "/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset.csv",
    ]

    print("[데이터 로드 시작]")
    df_list = []
    for file in file_paths:
        print(f"파일 로드 중: {file}")
        df = pd.read_csv(file)
        df_list.append(df)
        print(f"파일 로드 완료: {file} ({len(df)}개 행)")

    concatenated_data = pd.concat(df_list)
    print(
        f"[데이터 로드 완료] 총 {len(concatenated_data)}개 행, {concatenated_data.shape[1]}개 열"
    )

    # 데이터프레임을 딕셔너리 리스트로 변환
    data_list = concatenated_data.to_dict("records")

    # 번역 실행
    print("[번역 프로세스 시작]")
    translated_data = translate_english_to_korean(data_list)

    # JSON 형식으로 변환하여 출력 및 저장
    translated_json = json.dumps(translated_data, ensure_ascii=False, indent=2)

    print("\n[결과 저장 중]")
    output_file = "translated_output.json"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(translated_json)

    print(
        f"\n[작업 완료] {len(translated_data)}개의 데이터가 번역되어 '{output_file}' 파일로 저장되었습니다."
    )


if __name__ == "__main__":
    main()

[데이터 로드 시작]
파일 로드 중: /Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset_100.csv
파일 로드 완료: /Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset_100.csv (100개 행)
파일 로드 중: /Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset_ko_en_mixed.csv
파일 로드 완료: /Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset_ko_en_mixed.csv (300개 행)
파일 로드 중: /Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset.csv
파일 로드 완료: /Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset.csv (158개 행)
[데이터 로드 완료] 총 558개 행, 4개 열
[번역 프로세스 시작]

[번역 시작] 총 558개 항목 처리 예정


번역 진행률:   0%|          | 0/558 [00:00<?, ?it/s]


[번역 시작 - 항목 0]


처리 중: 0:   0%|          | 1/558 [00:09<1:30:39,  9.77s/it] 

[번역 완료 - 항목 0] (9.77초 소요)
{'user_input': '도시철도법시행령에 따른 운임조정 절차는 어떻게 진행되나요?', 'reference_contexts': "['운임조정 절차\\n\\n운임조정절차로 순서에 따른 내용 및 담당 제공\\n\\n순서 내용 담당 ① 운임 상한범위 조정 요청 운임조정의 필요성 등을 포함하여 부산시에 운임 상한범위 조정 요청 ② 운임조정 상한범위 결정 교통혁신위원회 및 물가대책위원회 의결 → 운임조정 상한액 결정 ③ 운임결정 및 신고 이사회 의결 및 부산시에 신고 ④ 운임조정액 확정 부산시에서 신고된 운임조정액 확정 ⑤ 운임조정 통보 기획재정부 및 국토교통부 에 통보\\n\\n근거법령\\n\\n도시철도법 제31조(운임의 신고 등)\\n\\n도시철도운영자는 운임변경 시 시·도지사가 정하는 범위 안에서 운임을 정하여 시·도지사에게 신고\\n\\n도시철도법시행령 제22조(운임의 조정 및 협의 등)']", 'reference': '도시철도법시행령 제22조에 근거하여 운임조정 절차는 다음과 같이 진행됩니다. 첫째, 운임 상한범위 조정 요청을 통해 운임조정의 필요성 등을 포함하여 부산시에 요청합니다. 둘째, 교통혁신위원회 및 물가대책위원회 의결을 거쳐 운임조정 상한액을 결정합니다. 셋째, 이사회 의결을 통해 운임을 결정하고 부산시에 신고합니다. 넷째, 부산시에서 신고된 운임조정액을 확정합니다. 마지막으로, 확정된 운임조정액을 기획재정부 및 국토교통부에 통보합니다.', 'synthesizer_name': 'single_hop_specifc_query_synthesizer'}

[번역 시작 - 항목 1]


...

[번역 완료 - 항목 79] (18.39초 소요)
{'user_input': '안녕하세요, 가족사랑카드가 다자녀 가구를 위한 부산 지하철 할인 혜택을 어떻게 제공하는지, 그리고 이 가족사랑카드의 다양한 종류와 발급 방법, 교통 기능이 있는지 알려주실 수 있나요?', 'reference_contexts': "['<1-hop>\\n\\n정기권 카드번호가 입력칸보다 작은 경우: 앞에 111을 입력 후 이어서 카드번호 입력\\n\\n9강 다자녀 운임할인\\n\\n다자녀 운임할인 ※ 동해남부선 적용 안됨\\n\\n대 상 자 : 부산시 거주(주민등록 기준), 자녀가 세 명 이상\\n\\n유효기간 : 막내 자녀가 만18세 까지\\n\\n할인조건 : 부산시에서 발급한 가족사랑카드에 등재된 자로 카드를 소지\\n\\n할인운임 : 어른운임의 50% 이내 할인\\n\\n구분 교통카드 QR승차권 1구간 2구간 1구간 2구간 운임 750원 850원 800원 900원\\n\\n다자녀 할인이용방법\\n\\n다자녀승차권 : 역사 승차권 자동발매기에서 다자녀승차권 발매·사용\\n\\n가족사랑카드', '<2-hop>\\n\\n가족사랑카드\\n\\n| 구분 | 부산시(신분확인용) | | 신한은행 신용(체크)카드 | | --- | --- | --- | --- | | | 모바일 카드 | 플라스틱 카드 | | | 발급대상 | 만13세 이상으로 휴대폰 소지자 | 모바일카드 신청이 불가능한 경우 | 신용카드 : 만19세 이상 체크카드 : 만13세 이상 | | 발급방법 | B-PASS 앱에서 즉시 발급 | 주민센터 방문신청, 신분증 지참 | 지점방문 또는 ARS(080-700-2108) | | 교통카드기능 | 없음 (모바일에서 cashbee 충전 후 사용) | 없음 (QR승차권 사용) | 후불 (신분확인 겸용) | | 시행시기 | 2022.4.4. | - | 신용카드 : 2021.1.30. 체크카드 : 2021.2.22. |\\n\\n게이트 경광등 노란색 표출,\\n\\n다자녀 기간 만료 시(막내 자녀 

처리 중: 80:  15%|█▍        | 81/558 [13:39<2:10:20, 16.40s/it]

[번역 완료 - 항목 80] (16.38초 소요)
{'user_input': '부산구간 도시철도와 동해선 간의 환승에 대한 요금 세부사항, 10km 초과 거리의 추가 요금 포함', 'reference_contexts': "['<1-hop>\\n\\n환승방법 대상 도시철도 1구간(원) 타교통수단(원) 총운임(원) 비고 도시철도→동해선(부산구간) 어른 1,600 200/10km 1,600(기본운임)+추가운임(부산구간) 도시철도 2구간 운임(어른 200, 청소년 100, 어린이 0원)은 하차 시 별도 부과 도시철도→동해선(부산구간) 청소년 1,050 150/10km 1,050(기본운임)+추가운임(부산구간) 추가운임(동해선)은 부산구간 10km 초과 시 10km마다 부과하는 운임 도시철도→동해선(부산구간) 어린이 무료 650+100/10km 650(기본운임)+추가운임(부산구간) 동해선(부산구간)→도시철도 어른 150 1,450+200/10km 1,600(기본운임)+추가운임(부산구간) 도시철도 2구간 운임(어른 200, 청소년 100, 어린이 0원)은 하차 시 별도 부과 동해선(부산구간)→도시철도 청소년 0 1,050+150/10km 1,050(기본운임)+추가운임(부산구간)', '<2-hop>\\n\\n동해선(부산구간)→도시철도 청소년 0 1,050+150/10km 1,050(기본운임)+추가운임(부산구간) 추가운임(동해선)은 부산구간 10km 초과 시 10km마다 부과하는 운임 동해선(부산구간)→도시철도 어린이 0 650+100/10km 650(기본운임)+추가운임(부산구간)']", 'reference': '부산구간 도시철도와 동해선 간의 환승에 대한 요금 세부사항은 다음과 같습니다: 도시철도에서 동해선(부산구간)으로 환승하는 경우, 어른의 기본운임은 1,600원이며, 초기 10km를 초과하는 경우 10km마다 200원의 추가 요금이 부과됩니다. 청소년의 경우 기본운임은 1,050원이며, 10km마다 150원의 추가 요금이 부과됩니다. 어린이는 기본운임이 650원이며, 

처리 중: 80:  15%|█▍        | 81/558 [19:39<1:55:47, 14.57s/it]


KeyboardInterrupt: 

In [8]:
def translate_item(client, item, index, pbar=None):
    """단일 항목을 번역하는 함수"""
    try:
        # 번역 요청 구성
        system_prompt = """당신은 영어를 한국어로 번역하는 전문가입니다.
        다음은 지하철역의 각종 업무에 대한 질문과 답변 데이터입니다.
주어진 영어 텍스트를 자연스러운 한국어로 번역해주세요.
전문 용어나 고유명사는 적절히 처리해주세요."""
        user_prompt = f"""다음 영어 텍스트를 한국어로 번역해주세요:
        출력은 JSON 형식으로만 추출해야하며 입력과 동일한 양식으로 출력해주세요.
[입력]
{json.dumps(item, ensure_ascii=False)}
번역 결과만 제공해주세요."""

        # 로그 출력
        print(f"\n[번역 시작 - 항목 {index}]")
        start_time = time.time()

        # API 호출로 번역
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.1,
            response_format={"type": "json_object"},
        )

        # 번역 결과 적용
        translated_text = response.choices[0].message.content.strip()
        translated_item = json.loads(translated_text)

        # 소요시간 및 완료 로그
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"[번역 완료 - 항목 {index}] ({elapsed_time:.2f}초 소요)")

        # 진행바 업데이트
        if pbar:
            pbar.update(1)
            pbar.set_description(f"처리 중: {index}")

        return translated_item

    except Exception as e:
        error_msg = f"Error occurred at index {index}: {e}"
        print(f"\n[에러 - 항목 {index}] {error_msg}")

        # 진행바 업데이트
        if pbar:
            pbar.update(1)
            pbar.set_description(f"에러 발생: {index}")

        return None


def translate_english_to_korean(data_list, output_path="translated_output.jsonl"):
    """
    List[dict] 형태의 데이터에서 영어로 된 'user_input'과 'reference' 필드를 한국어로 번역합니다.
    각 항목을 개별적으로 API 호출하여 번역합니다.
    번역 결과는 매번 output_path에 append 저장합니다.
    이미 번역된 인덱스는 건너뜁니다.
    """
    from openai import OpenAI

    client = OpenAI()

    # 이미 번역된 데이터 읽기
    translated_indices = set()
    translated_data = []
    if os.path.exists(output_path):
        with open(output_path, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    item = json.loads(line)
                    translated_data.append(item)
                    if "index" in item:
                        translated_indices.add(item["index"])
                except Exception:
                    continue

    for index, item in enumerate(data_list):
        if index in translated_indices:
            continue  # 이미 번역된 항목은 건너뜀

        try:
            system_prompt = """당신은 영어를 한국어로 번역하는 전문가입니다.
            다음은 지하철역의 각종 업무에 대한 질문과 답변 데이터입니다.
주어진 영어 텍스트를 자연스러운 한국어로 번역해주세요.
전문 용어나 고유명사는 적절히 처리해주세요."""
            user_prompt = f"""다음 영어 텍스트를 한국어로 번역해주세요:
출력은 JSON 형식으로만 추출해야하며 입력과 동일한 양식으로 출력해주세요.

[입력]
{json.dumps(item, ensure_ascii=False)}

번역 결과만 제공해주세요."""

            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt},
                ],
                temperature=0.1,
                response_format={"type": "json_object"},
            )
            translated_text = response.choices[0].message.content.strip()
            translated_item = json.loads(translated_text)
            translated_item["index"] = index  # 인덱스 정보 추가

            # 결과를 파일에 append
            with open(output_path, "a", encoding="utf-8") as f:
                f.write(json.dumps(translated_item, ensure_ascii=False) + "\n")

            translated_data.append(translated_item)
            print(f"✅ 번역 완료: {index + 1}/{len(data_list)}")
        except Exception as e:
            print(f"❌ Error at index {index}: {e}")
            continue

    return translated_data


def main():
    file_paths = [
        "/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset_100.csv",
        "/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset_ko_en_mixed.csv",
        "/Users/sdh/Dev/projects/humetro-ai-assistant/datasets/synthetic_qa_dataset.csv",
    ]
    df_list = []
    for file in file_paths:
        df = pd.read_csv(file)
        df_list.append(df)
    concatenated_data = pd.concat(df_list)
    print(concatenated_data.shape)

    data_list = concatenated_data.to_dict("records")
    translated_data = translate_english_to_korean(
        data_list, output_path="translated_output.jsonl"
    )

    print(
        f"\n전체 번역 완료! {len(translated_data)}개 데이터가 번역되었습니다. 'translated_output.jsonl' 파일을 확인하세요."
    )


In [ ]:
if __name__ == "__main__":
    main()

(558, 4)
✅ 번역 완료: 30/558
✅ 번역 완료: 31/558
✅ 번역 완료: 32/558
✅ 번역 완료: 33/558
✅ 번역 완료: 34/558
✅ 번역 완료: 35/558
✅ 번역 완료: 36/558
✅ 번역 완료: 37/558
✅ 번역 완료: 38/558
✅ 번역 완료: 39/558
✅ 번역 완료: 40/558
✅ 번역 완료: 41/558
✅ 번역 완료: 42/558
✅ 번역 완료: 43/558
✅ 번역 완료: 44/558
✅ 번역 완료: 45/558
✅ 번역 완료: 46/558
✅ 번역 완료: 47/558
✅ 번역 완료: 48/558
✅ 번역 완료: 49/558
✅ 번역 완료: 50/558
✅ 번역 완료: 51/558
✅ 번역 완료: 52/558
✅ 번역 완료: 53/558
✅ 번역 완료: 54/558
✅ 번역 완료: 55/558
✅ 번역 완료: 56/558
✅ 번역 완료: 57/558
✅ 번역 완료: 58/558
✅ 번역 완료: 59/558
✅ 번역 완료: 60/558
✅ 번역 완료: 61/558
✅ 번역 완료: 62/558
✅ 번역 완료: 63/558
✅ 번역 완료: 64/558
✅ 번역 완료: 65/558
✅ 번역 완료: 66/558
✅ 번역 완료: 67/558
✅ 번역 완료: 68/558
✅ 번역 완료: 69/558
✅ 번역 완료: 70/558
✅ 번역 완료: 71/558
✅ 번역 완료: 72/558
✅ 번역 완료: 73/558
✅ 번역 완료: 74/558
✅ 번역 완료: 75/558
✅ 번역 완료: 76/558
✅ 번역 완료: 77/558
✅ 번역 완료: 78/558
✅ 번역 완료: 79/558
✅ 번역 완료: 80/558
✅ 번역 완료: 81/558
✅ 번역 완료: 82/558
✅ 번역 완료: 83/558
✅ 번역 완료: 84/558
✅ 번역 완료: 85/558
✅ 번역 완료: 86/558
✅ 번역 완료: 87/558
